In [1]:
# Tratamiento de datos
# -----------------------------------------------------------------------
import pandas as pd
import numpy as np
import os
from unidecode import unidecode
# Visualización
# ------------------------------------------------------------------------------
import matplotlib.pyplot as plt
import seaborn as sns

# Evaluar linealidad de las relaciones entre las variables
# y la distribución de las variables
# ------------------------------------------------------------------------------
import scipy.stats as stats
from scipy.stats import ttest_ind, norm, chi2_contingency, f_oneway, shapiro, kstest
from sklearn.linear_model import LinearRegression

# Configuración
# -----------------------------------------------------------------------
pd.set_option('display.max_columns', None) # para poder visualizar todas las columnas de los DataFrames
pd.options.display.float_format = None

# Gestión de los warnings
# -----------------------------------------------------------------------
import warnings
warnings.filterwarnings("ignore")

In [3]:
comex = pd.read_csv('Files/DataComex_20240419_113058.csv', encoding = "ISO-8859-1", sep=";")

In [4]:
comex.head(5)

,flujo_codigo,flujo_nombre,periodo_anio,periodo_mes,pais_codigo,pais_nombre,taric,euros,kilos,Unnamed: 9
0,I,Importación,2022,NaN,660,Afganistán,3,85850,14291,NaN
1,I,Importación,2022,NaN,660,Afganistán,7,3402,3743,NaN
2,I,Importación,2022,NaN,660,Afganistán,8,"603337,78",117520,NaN
3,I,Importación,2022,NaN,660,Afganistán,9,"2143995,55","2604,7",NaN
4,I,Importación,2021,NaN,660,Afganistán,4,"12428,8",320,NaN


In [5]:
# Directorio donde se encuentran los archivos CSV
directorio = 'Files/'

# Obtener la lista de archivos CSV en el directorio
archivos_csv = [archivo for archivo in os.listdir(directorio) if archivo.endswith('.csv')]

# Inicializar una lista para almacenar todos los DataFrames
dataframes = []

# Iterar sobre cada archivo CSV y leerlo en un DataFrame, luego agregarlo a la lista
for archivo in archivos_csv:
    df = pd.read_csv(os.path.join(directorio, archivo), encoding="ISO-8859-1", sep=";")
    dataframes.append(df)

# Concatenar todos los DataFrames en uno solo
df_final = pd.concat(dataframes, ignore_index=True)

# Ahora, df_final contiene toda la información de los diferentes archivos CSV combinados en un único DataFrame

# Puedes imprimir el DataFrame final
print(df_final.head(5))


  flujo_codigo flujo_nombre  periodo_anio  periodo_mes  pais_codigo  \
0            I  Importación          2022          NaN          660   
1            I  Importación          2022          NaN          660   
2            I  Importación          2022          NaN          660   
3            I  Importación          2022          NaN          660   
4            I  Importación          2021          NaN          660   

  pais_nombre  taric       euros   kilos  Unnamed: 9  
0  Afganistán      3       85850   14291         NaN  
1  Afganistán      7        3402    3743         NaN  
2  Afganistán      8   603337,78  117520         NaN  
3  Afganistán      9  2143995,55  2604,7         NaN  
4  Afganistán      4     12428,8     320         NaN  


In [6]:
df_final = df_final.drop(["periodo_mes","Unnamed: 9"],axis=1)

In [7]:
df_final["euros"] = df_final["euros"].str.replace(",", ".")
df_final["kilos"] = df_final["kilos"].str.replace(",", ".")

In [8]:
df_final["euros"] = df_final["euros"].astype(float)
df_final["kilos"] = df_final["kilos"].astype(float)

In [9]:
df_final.dtypes

flujo_codigo     object
flujo_nombre     object
periodo_anio      int64
pais_codigo       int64
pais_nombre      object
taric             int64
euros           float64
kilos           float64
dtype: object

In [10]:
df_final["pais_nombre"].unique()

array(['Afganistán', 'Albania', 'Alemania', 'Andorra', 'Angola',
       'Antigua y Barbuda', 'Arabia Saudí', 'Argelia', 'Argentina',
       'Armenia', 'Australia', 'Austria', 'Azerbaiján', 'Bahamas',
       'Bahrein', 'Bangladesh', 'Bélgica', 'Belice', 'Benin',
       'Bielorusia', 'Bolivia', 'Bonaire, San Eustaquio y Saba',
       'Bosnia-Herzegovina', 'Botswana', 'Brasil', 'Brunei Darussalam',
       'Bulgaria', 'Burkina Faso (ant. Alto Volta)', 'Burundi',
       'Cabo Verde', 'Camboya (Campuchea)', 'Camerún', 'Canadá', 'Chile',
       'China', 'Chipre', 'Cisjordania/Franja de Gaza',
       'Ciudad del Vaticano', 'Colombia', 'Comoras', 'Congo',
       'Corea del Norte (Rep.Pop.Dem.de Corea)',
       'Corea del Sur (Rep. de Corea)', 'Costa de Marfil', 'Costa Rica',
       'Croacia', 'Cuba', 'Curaçao', 'Dinamarca', 'Djibouti', 'Ecuador',
       'Egipto', 'El Salvador', 'Emiratos Árabes Unidos', 'Eslovaquia',
       'Eslovenia', 'Estados Unidos', 'Estonia', 'Etiopía', 'Filipinas',
     

In [11]:
paises_continente = {
    'Afganistán': 'Asia',
    'Albania': 'Europa',
    'Alemania': 'Europa',
    'Andorra': 'Europa',
    'Angola': 'África',
    'Antigua y Barbuda': 'América del Norte',
    'Argentina': 'América del Sur',
    'Armenia': 'Asia',
    'Australia': 'Oceanía',
    'Austria': 'Europa',
    'Azerbaiyán': 'Asia',
    'Bahamas': 'América del Norte',
    'Bahréin': 'Asia',
    'Bangladesh': 'Asia',
    'Barbados': 'América del Norte',
    'Bielorrusia': 'Europa',
    'Bélgica': 'Europa',
    'Belice': 'América del Norte',
    'Benín': 'África',
    'Bután': 'Asia',
    'Bolivia': 'América del Sur',
    'Bosnia y Herzegovina': 'Europa',
    'Botsuana': 'África',
    'Brasil': 'América del Sur',
    'Brunéi': 'Asia',
    'Bulgaria': 'Europa',
    'Burkina Faso': 'África',
    'Burundi': 'África',
    'Cabo Verde': 'África',
    'Camboya': 'Asia',
    'Camerún': 'África',
    'Canadá': 'América del Norte',
    'Chad': 'África',
    'Chile': 'América del Sur',
    'China': 'Asia',
    'Chipre': 'Europa',
    'Colombia': 'América del Sur',
    'Comoras': 'África',
    'Corea del Norte': 'Asia',
    'Corea del Sur': 'Asia',
    'Costa de Marfil': 'África',
    'Costa Rica': 'América del Norte',
    'Croacia': 'Europa',
    'Cuba': 'América del Norte',
    'Dinamarca': 'Europa',
    'Dominica': 'América del Norte',
    'Ecuador': 'América del Sur',
    'Egipto': 'África',
    'El Salvador': 'América del Norte',
    'Emiratos Árabes Unidos': 'Asia',
    'Eritrea': 'África',
    'Eslovaquia': 'Europa',
    'Eslovenia': 'Europa',
    'España': 'Europa',
    'Estados Unidos': 'América del Norte',
    'Estonia': 'Europa',
    'Etiopía': 'África',
    'Fiyi': 'Oceanía',
    'Filipinas': 'Asia',
    'Finlandia': 'Europa',
    'Francia': 'Europa',
    'Gabón': 'África',
    'Gambia': 'África',
    'Georgia': 'Asia',
    'Ghana': 'África',
    'Granada': 'América del Norte',
    'Grecia': 'Europa',
    'Guatemala': 'América del Norte',
    'Guinea': 'África',
    'Guinea-Bisáu': 'África',
    'Guinea Ecuatorial': 'África',
    'Guyana': 'América del Sur',
    'Haití': 'América del Norte',
    'Honduras': 'América del Norte',
    'Hungría': 'Europa',
    'India': 'Asia',
    'Indonesia': 'Asia',
    'Irak': 'Asia',
    'Irán': 'Asia',
    'Irlanda': 'Europa',
    'Islandia': 'Europa',
    'Islas Marshall': 'Oceanía',
    'Islas Salomón': 'Oceanía',
    'Israel': 'Asia',
    'Italia': 'Europa',
    'Jamaica': 'América del Norte',
    'Japón': 'Asia',
    'Jordania': 'Asia',
    'Kazajistán': 'Asia',
    'Kenia': 'África',
    'Kirguistán': 'Asia',
    'Kiribati': 'Oceanía',
    'Kuwait': 'Asia',
    'Laos': 'Asia',
    'Lesoto': 'África',
    'Letonia': 'Europa',
    'Líbano': 'Asia',
    'Liberia': 'África',
    'Libia': 'África',
    'Liechtenstein': 'Europa',
    'Lituania': 'Europa',
    'Luxemburgo': 'Europa',
    'Madagascar': 'África',
    'Malasia': 'Asia',
    'Malaui': 'África',
    'Maldivas': 'Asia',
    'Malí': 'África',
    'Malta': 'Europa',
    'Marruecos': 'África',
    'Mauricio': 'África',
    'Mauritania': 'África',
    'México': 'América del Norte',
    'Micronesia': 'Oceanía',
    'Moldavia': 'Europa',
    'Mónaco': 'Europa',
    'Mongolia': 'Asia',
    'Montenegro': 'Europa',
    'Mozambique': 'África',
    'Namibia': 'África',
    'Nauru': 'Oceanía',
    'Nepal': 'Asia',
    'Nicaragua': 'América del Norte',
    'Níger': 'África',
    'Nigeria': 'África',
    'Noruega': 'Europa',
    'Nueva Zelanda': 'Oceanía',
    'Omán': 'Asia',
    'Países Bajos': 'Europa',
    'Pakistán': 'Asia',
    'Palaos': 'Oceanía',
    'Panamá': 'América del Norte',
    'Papúa Nueva Guinea': 'Oceanía',
    'Paraguay': 'América del Sur',
    'Perú': 'América del Sur',
    'Polonia': 'Europa',
    'Portugal': 'Europa',
    'Qatar': 'Asia',
    'Reino Unido': 'Europa',
    'República Centroafricana': 'África',
    'República Checa': 'Europa',
    'República del Congo': 'África',
    'República Democrática del Congo': 'África',
    'República Dominicana': 'América del Norte',
    'República Sudafricana': 'África',
    'Ruanda': 'África',
    'Rumania': 'Europa',
    'Rusia': 'Europa',
    'Samoa': 'Oceanía',
    'San Cristóbal y Nieves': 'América del Norte',
    'San Marino': 'Europa',
    'San Vicente y las Granadinas': 'América del Norte',
    'Santa Lucía': 'América del Norte',
    'Santo Tomé y Príncipe': 'África',
    'Senegal': 'África',
    'Serbia': 'Europa',
    'Seychelles': 'África',
    'Sierra Leona': 'África',
    'Singapur': 'Asia',
    'Siria': 'Asia',
    'Somalia': 'África',
    'Sri Lanka': 'Asia',
    'Suazilandia': 'África',
    'Sudán': 'África',
    'Sudán del Sur': 'África',
    'Suecia': 'Europa',
    'Suiza': 'Europa',
    'Surinam': 'América del Sur',
    'Tailandia': 'Asia',
    'Tanzania': 'África',
    'Tayikistán': 'Asia',
    'Timor Oriental': 'Asia',
    'Togo': 'África',
    'Tonga': 'Oceanía',
    'Trinidad y Tobago': 'América del Norte',
    'Túnez': 'África',
    'Turkmenistán': 'Asia',
    'Turquía': 'Asia',
    'Tuvalu': 'Oceanía',
    'Ucrania': 'Europa',
    'Uganda': 'África',
    'Uruguay': 'América del Sur',
    'Uzbekistán': 'Asia',
    'Vanuatu': 'Oceanía',
    'Vaticano': 'Europa',
    'Venezuela': 'América del Sur',
    'Vietnam': 'Asia',
    'Yemen': 'Asia',
    'Yibuti': 'África',
    'Zambia': 'África',
    'Zimbabue': 'África',
    'Arabia Saudí': 'Asia',
    'Argelia': 'África',
    'Azerbaiyán': 'Asia',
    'Bahrein': 'Asia',
    'Benin': 'África',
    'Bielorusia': 'Europa',
    'Bonaire, San Eustaquio y Saba': 'América del Norte',
    'Bosnia-Herzegovina': 'Europa',
    'Botswana': 'África',
    'Brunei Darussalam': 'Asia',
    'Burkina Faso (ant. Alto Volta)': 'África',
    'Camboya (Campuchea)': 'Asia',
    'Cisjordania/Franja de Gaza': 'Asia',
    'Ciudad del Vaticano': 'Europa',
    'Congo': 'África',
    'Corea del Norte (Rep.Pop.Dem.de Corea)': 'Asia',
    'Corea del Sur (Rep. de Corea)': 'Asia',
    'Curaçao': 'América del Norte',
    'Djibouti': 'África',
    'Groenlandia': 'América del Norte',
    'Guinea Bissau': 'África',
    'Hong-Kong': 'Asia',
    'Islas Caimán': 'América del Norte',
    'Islas Cocos o islas Keeling': 'Asia',
    'Islas Feroe': 'Europa',
    'Islas Fiyi(Fidji)': 'Oceanía',
    'Islas Maldivas': 'Asia',
    'Islas Malvinas (Falkland)': 'América del Sur',
    'Islas Salomon': 'Oceanía',
    'Islas Tokelau': 'Oceanía',
    'Kazajstán': 'Asia',
    'Lesotho': 'África',
    'Macao': 'Asia',
    'Macedonia del Norte': 'Europa',
    'Malawi': 'África',
    'Mali': 'África',
    'Myanmar': 'Asia',
    'Nueva Caledonia y dependencias': 'Oceanía',
    'Países y territorios no determinados': None,
    'Países y territorios no determinados.Extraco.': None,
    'Países y territorios no determinados.Intraco.': None,
    'Pitcairn': 'Oceanía',
    'Polinesia Francesa': 'Oceanía',
    'R.D. del Congo': 'África',
    'Rumanía': 'Europa',
    'Samoa Americana': 'Oceanía',
    'Sant Bartolomé': 'América del Norte',
    'Santa Elena': 'África',
    'Sudáfrica': 'África',
    'Taiwán': 'Asia',
    'Tajikistán': 'Asia',
    'Tierras Australes Francesas': 'Oceanía',
    'Timor-Leste': 'Asia',
    'Zimbabwe': 'África',
    'Aruba': 'América del Norte',
    'Avituall.y combust.intercambios comunitarios': None,
    'Avituallamiento terceros': None,
    'Bermudas': 'América del Norte',
    'Georgia del Sur e Islas Sandwich Sur': 'América del Sur',
    'Gibraltar': 'Europa',
    'Guam': 'Oceanía',
    'Islas Cook': 'Oceanía',
    'Islas Virgenes (GB)': 'América del Norte',
    'Islas Wallis y Futuna': 'Oceanía',
    'Kosovo': 'Europa',
    'San Martín': 'América del Norte',
    'San Vicente y Granadinas': 'América del Norte',
    'Alta mar fuera de las aguas territoriales': None,
    'Antártida': None,
    'Isla Bouvet': None,
    'Islas Marianas del Norte': 'Oceanía',
    'Islas Turcas y Caicos': 'América del Norte',
    'Islas Virgenes (EEUU)': 'América del Norte',
    'Palau': 'Oceanía',
    'Sáhara Occidental': 'África',
    'Samoa Occidental': 'Oceanía',
    'San Pedro y Miquelón': 'América del Norte',
    'Swazilandia': 'África',
    'Territorio Británico Indico': 'Asia',
    'Anguilla': 'América del Norte',
    'Avituallamiento y combustible': None,
    'Islas Menores alejadas Estados Unidos': 'América del Norte',
    'Micronesia Estados Federados': 'Oceanía',
    'Isla Navidad': 'Oceanía',
    'Melilla': 'África',
    'Montserrat': 'América del Norte',
    'Isla Niue': 'Oceanía',
    'Isla Norfolk': 'Oceanía',
    'Países y territ.no prec.raz.com.Intra.;Código País Especial': None,
    'Isla Heard e Islas McDonald': None,
    'Países y territ.no prec.raz.com.Extra.': None,
    'Azerbaiján': 'Asia',
    'Antártida':'Antartida',
    'Isla Bouvet': 'Antartida',
    'Isla Heard e Islas McDonald':'Oceania' 
}


In [12]:
df_final['continente'] = df_final['pais_nombre'].map(paises_continente)

In [13]:
df_final.head(5)

,flujo_codigo,flujo_nombre,periodo_anio,pais_codigo,pais_nombre,taric,euros,kilos,continente
0,I,Importación,2022,660,Afganistán,3,85850.00,14291.0,Asia
1,I,Importación,2022,660,Afganistán,7,3402.00,3743.0,Asia
2,I,Importación,2022,660,Afganistán,8,603337.78,117520.0,Asia
3,I,Importación,2022,660,Afganistán,9,2143995.55,2604.7,Asia
4,I,Importación,2021,660,Afganistán,4,12428.80,320.0,Asia


In [14]:
taric = pd.read_csv('Taric.csv', encoding = "ISO-8859-1", sep=";")

In [15]:
taric

,Partida,Descripción
0,0,COMERCIO CONFIDENCIAL
1,1,ANIMALES VIVOS
2,2,CARNE Y DESPOJOS COMESTIBLES
3,3,"PESCADOS Y CRUSTÁCEOS, MOLUSCOS Y DEMÁS INVERT..."
4,4,LECHE Y PRODUCTOS LÁCTEOS; HUEVOS DE AVE; MIEL...
...,...,...
91,92,INSTRUMENTOS MUSICALES; SUS PARTES Y ACCESORIOS
92,93,"ARMAS, MUNICIONES, Y SUS PARTES Y ACCESORIOS"
93,94,MUEBLES; MOBILIARIO MEDICOQUIRÚRGICO; ARTÍCULO...
94,95,"JUGUETES, JUEGOS Y ARTÍCULOS PARA RECREO O DEP..."


In [16]:
df_merged = pd.merge(df_final, taric, left_on='taric', right_on='Partida', how='inner')

In [17]:
df_merged['Descripción'] = df_merged['Descripción'].str.lower()

In [18]:
data = df_merged

In [19]:
data.head(5)

,flujo_codigo,flujo_nombre,periodo_anio,pais_codigo,pais_nombre,taric,euros,kilos,continente,Partida,Descripción
0,I,Importación,2022,660,Afganistán,3,85850.00,14291.00,Asia,3,"pescados y crustáceos, moluscos y demás invert..."
1,I,Importación,2022,70,Albania,3,5993990.95,506100.36,Europa,3,"pescados y crustáceos, moluscos y demás invert..."
2,I,Importación,2021,70,Albania,3,7270432.96,721024.55,Europa,3,"pescados y crustáceos, moluscos y demás invert..."
3,I,Importación,2020,70,Albania,3,7583890.54,772762.58,Europa,3,"pescados y crustáceos, moluscos y demás invert..."
4,I,Importación,2019,70,Albania,3,6854518.30,697792.62,Europa,3,"pescados y crustáceos, moluscos y demás invert..."


In [20]:
data = data.dropna()

In [21]:
for columna in data.columns:
    # Verificar si el tipo de dato de la columna es de texto
    if data[columna].dtype == 'object':
        # Aplicar unidecode a cada celda de texto en la columna
        data[columna] = data[columna].apply(unidecode)

In [22]:
data

,flujo_codigo,flujo_nombre,periodo_anio,pais_codigo,pais_nombre,taric,euros,kilos,continente,Partida,Descripción
0,I,Importacion,2022,660,Afganistan,3,8.585000e+04,14291.00,Asia,3,"pescados y crustaceos, moluscos y demas invert..."
1,I,Importacion,2022,70,Albania,3,5.993991e+06,506100.36,Europa,3,"pescados y crustaceos, moluscos y demas invert..."
2,I,Importacion,2021,70,Albania,3,7.270433e+06,721024.55,Europa,3,"pescados y crustaceos, moluscos y demas invert..."
3,I,Importacion,2020,70,Albania,3,7.583891e+06,772762.58,Europa,3,"pescados y crustaceos, moluscos y demas invert..."
4,I,Importacion,2019,70,Albania,3,6.854518e+06,697792.62,Europa,3,"pescados y crustaceos, moluscos y demas invert..."
...,...,...,...,...,...,...,...,...,...,...,...
117178,E,Exportacion,2023,52,Turquia,93,4.123296e+06,139902.04,Asia,93,"armas, municiones, y sus partes y accesorios"
117179,E,Exportacion,2023,72,Ucrania,93,1.830749e+08,5505574.48,Europa,93,"armas, municiones, y sus partes y accesorios"
117180,E,Exportacion,2023,524,Uruguay,93,5.266365e+05,25597.03,America del Sur,93,"armas, municiones, y sus partes y accesorios"
117181,E,Exportacion,2023,81,Uzbekistan,93,5.505100e+04,5376.00,Asia,93,"armas, municiones, y sus partes y accesorios"


In [23]:
data.to_csv("data_pais.csv")

In [3]:
comex2 = pd.read_csv('Files_detalle/DataComex_20240428_190224.csv', encoding = "ISO-8859-1", sep=";")

In [5]:
# Directorio donde se encuentran los archivos CSV
directorio = 'Files_detalle/'

# Obtener la lista de archivos CSV en el directorio
archivos_csv = [archivo for archivo in os.listdir(directorio) if archivo.endswith('.csv')]

# Inicializar una lista para almacenar todos los DataFrames
dataframes = []

# Iterar sobre cada archivo CSV y leerlo en un DataFrame, luego agregarlo a la lista
for archivo in archivos_csv:
    df = pd.read_csv(os.path.join(directorio, archivo), encoding="ISO-8859-1", sep=";")
    dataframes.append(df)

# Concatenar todos los DataFrames en uno solo
df_detalle = pd.concat(dataframes, ignore_index=True)

# Ahora, df_final contiene toda la información de los diferentes archivos CSV combinados en un único DataFrame

# Puedes imprimir el DataFrame final
print(df_detalle.head(5))


  flujo_codigo flujo_nombre  periodo_anio  periodo_mes  pais_codigo  \
0            I  Importación          2023          NaN          660   
1            I  Importación          2023          NaN          660   
2            I  Importación          2023          NaN          660   
3            I  Importación          2022          NaN          660   
4            I  Importación          2022          NaN          660   

  pais_nombre  taric      euros   kilos  Unnamed: 9  
0  Afganistán      8  518439,73  120250         NaN  
1  Afganistán    802  515889,73  117250         NaN  
2  Afganistán    806       2550    3000         NaN  
3  Afganistán      8  603337,78  117520         NaN  
4  Afganistán    802  603337,78  117520         NaN  


In [7]:
df_detalle = df_detalle.drop(["periodo_mes","Unnamed: 9"],axis=1)

In [8]:
df_detalle["euros"] = df_detalle["euros"].str.replace(",", ".")
df_detalle["kilos"] = df_detalle["kilos"].str.replace(",", ".")

In [9]:
df_detalle["euros"] = df_detalle["euros"].astype(float)
df_detalle["kilos"] = df_detalle["kilos"].astype(float)

In [10]:
df_detalle.head(5)

,flujo_codigo,flujo_nombre,periodo_anio,pais_codigo,pais_nombre,taric,euros,kilos
0,I,Importación,2023,660,Afganistán,8,518439.73,120250.0
1,I,Importación,2023,660,Afganistán,802,515889.73,117250.0
2,I,Importación,2023,660,Afganistán,806,2550.00,3000.0
3,I,Importación,2022,660,Afganistán,8,603337.78,117520.0
4,I,Importación,2022,660,Afganistán,802,603337.78,117520.0


In [11]:
df_detalle.dtypes

flujo_codigo     object
flujo_nombre     object
periodo_anio      int64
pais_codigo       int64
pais_nombre      object
taric             int64
euros           float64
kilos           float64
dtype: object

In [12]:
def quitar_notacion_cientifica(numero):
    numero_decimal = float(numero)  # Convertir la cadena a un número
    numero_redondeado = round(numero_decimal, 2)  # Redondear el número a 2 decimales
    return '{:.2f}'.format(numero_redondeado)  # Formatear el número redondeado como cadena sin notación científica

In [13]:
columnas_a_convertir = ['euros', 'kilos']
for columna in columnas_a_convertir:
    df_detalle[columna] = df_detalle[columna].apply(quitar_notacion_cientifica)

In [14]:
df_detalle.head(5)

,flujo_codigo,flujo_nombre,periodo_anio,pais_codigo,pais_nombre,taric,euros,kilos
0,I,Importación,2023,660,Afganistán,8,518439.73,120250.00
1,I,Importación,2023,660,Afganistán,802,515889.73,117250.00
2,I,Importación,2023,660,Afganistán,806,2550.00,3000.00
3,I,Importación,2022,660,Afganistán,8,603337.78,117520.00
4,I,Importación,2022,660,Afganistán,802,603337.78,117520.00


In [15]:
df_detalle = df_detalle.drop(df_detalle[df_detalle['taric'] == 8].index)


In [16]:
df_detalle.head(5)

,flujo_codigo,flujo_nombre,periodo_anio,pais_codigo,pais_nombre,taric,euros,kilos
1,I,Importación,2023,660,Afganistán,802,515889.73,117250.00
2,I,Importación,2023,660,Afganistán,806,2550.00,3000.00
4,I,Importación,2022,660,Afganistán,802,603337.78,117520.00
6,I,Importación,2021,660,Afganistán,802,550245.28,128390.00
8,I,Importación,2020,660,Afganistán,802,31028.56,970.00


In [17]:
frutas = pd.read_csv('taric_capitulo_8.csv', encoding = "ISO-8859-1", sep=";")

In [18]:
frutas

,Codigo,Descripción
0,801,"Cocos, nueces del Brasil y nueces de marañón (..."
1,802,"Los demás frutos de cáscara frescos o secos, i..."
2,803,"Plátanos (bananas), incluidos los ""plantains"" ..."
3,804,"Dátiles, higos, piñas (ananás), aguacates (pal..."
4,805,Agrios (cítricos) frescos o secos
5,806,"Uvas, frescas o secas, incluidas las pasas"
6,807,"Melones, sandías y papayas, frescos"
7,808,"Manzanas, peras y membrillos, frescos"
8,809,"Albaricoques (damascos, chabacanos), cerezas, ..."
9,810,"Las demás frutas u otros frutos, frescos"


In [19]:
df_frutas = pd.merge(df_detalle, frutas, left_on='taric', right_on='Codigo', how='inner')

In [21]:
df_frutas['Descripción'] = df_frutas['Descripción'].str.lower()

In [23]:
df_frutas.head(5)

,flujo_codigo,flujo_nombre,periodo_anio,pais_codigo,pais_nombre,taric,euros,kilos,Codigo,Descripción
0,I,Importación,2023,660,Afganistán,802,515889.73,117250.00,802,"los demás frutos de cáscara frescos o secos, i..."
1,I,Importación,2022,660,Afganistán,802,603337.78,117520.00,802,"los demás frutos de cáscara frescos o secos, i..."
2,I,Importación,2021,660,Afganistán,802,550245.28,128390.00,802,"los demás frutos de cáscara frescos o secos, i..."
3,I,Importación,2020,660,Afganistán,802,31028.56,970.00,802,"los demás frutos de cáscara frescos o secos, i..."
4,I,Importación,2019,660,Afganistán,802,188442.17,42184.00,802,"los demás frutos de cáscara frescos o secos, i..."


In [24]:
for columna in df_frutas.columns:
    # Verificar si el tipo de dato de la columna es de texto
    if df_frutas[columna].dtype == 'object':
        # Aplicar unidecode a cada celda de texto en la columna
        df_frutas[columna] = df_frutas[columna].apply(unidecode)

In [25]:
df_frutas

,flujo_codigo,flujo_nombre,periodo_anio,pais_codigo,pais_nombre,taric,euros,kilos,Codigo,Descripción
0,I,Importacion,2023,660,Afganistan,802,515889.73,117250.00,802,"los demas frutos de cascara frescos o secos, i..."
1,I,Importacion,2022,660,Afganistan,802,603337.78,117520.00,802,"los demas frutos de cascara frescos o secos, i..."
2,I,Importacion,2021,660,Afganistan,802,550245.28,128390.00,802,"los demas frutos de cascara frescos o secos, i..."
3,I,Importacion,2020,660,Afganistan,802,31028.56,970.00,802,"los demas frutos de cascara frescos o secos, i..."
4,I,Importacion,2019,660,Afganistan,802,188442.17,42184.00,802,"los demas frutos de cascara frescos o secos, i..."
...,...,...,...,...,...,...,...,...,...,...
7990,E,Exportacion,2023,39,Suiza,814,585326.47,52167.00,814,"cortezas de agrios (citricos), melones o sandi..."
7991,E,Exportacion,2022,39,Suiza,814,549780.40,54433.00,814,"cortezas de agrios (citricos), melones o sandi..."
7992,E,Exportacion,2021,39,Suiza,814,451871.80,36550.00,814,"cortezas de agrios (citricos), melones o sandi..."
7993,E,Exportacion,2020,39,Suiza,814,395520.40,44363.80,814,"cortezas de agrios (citricos), melones o sandi..."


In [26]:
df_frutas.to_csv("data_frutas.csv")